# Analyzing importance of game scores

NBA Analytics expert John Hollinger developed the stat Player Efficiency Rating, which measures the performance of a player in a game. This stat takes into normal individual stats, like rebounds and points, while also taking into account game flow (how many possessions are in the game). This way, players aren't penalized if their team less times per game, which would result in the player having fewer opportunities to accumulate stats.

John Hollinger also developed a simpler, linear version of Player Efficiency that doesn't take into account game flow. It measures the performance of a player in a single game. For each player, his Hollinger Game Score is:

$
PTS + 0.4 * FG - 0.7 * FGA - 0.4*(FTA - FT) + 0.7 * ORB + 0.3 * DRB + STL + 0.7 * AST + 0.7 * BLK - 0.4 * PF - TOV,
$

where each is an individual stat for the player. Here, the abbreviations are
- `PTS`, number of points scored.
- `FG`, field goals made.
- `FGA`, field goals attempted.
- `FTA`, free throws attempted.
- `FT`, free throws made.
- `ORB`, offensive rebounds.
- `DRB`, defensive rebounds.
- `STL`, steals.
- `AST`, assists.
- `BLK`, blocks.
- `PF`, personal fouls.
- `TOV`, turnovers.

You can read more about John Hollinger at https://en.wikipedia.org/wiki/John_Hollinger. His Wikipedia page says "The scale is similar to that of points scored, (40 is an outstanding performance, 10 is an average performance, etc.)."

We will be interested how well Hollinger Game Score predicts whether a team wins. More specifically, does the player with highest game score usually win? Does the team with average game score usually win?

We will start by just analyzing the Warriors 2017-2018 season.

In [1]:
% matplotlib inline


#necessary libraries
import numpy as np

import pandas as pd
pd.set_option('display.max_columns',None)
from pandas.util.testing import assert_frame_equal
pd.options.mode.chained_assignment = None  # default='warn'


import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib as mpl
mpl.rcParams.update({'axes.titlesize' : 20,
                     'axes.labelsize' : 18,
                     'legend.fontsize': 16})

# Set default seaborn plotting style
sns.set_style('white')

from datetime import datetime
import time
from nose.tools import assert_equal

import sqlite3

We need to gather the player stats for every Warriors game during the season- both their stats and their opponents' stats. We will follow the method from last notebook for doing this. We will first list all of the games played by the Warriors, extracting from the file 'all_games_04_on.csv'. We then use a SQL join to attach the player stats for each game.

In [2]:
#find games by Matchup ID for Warriors season#find g 
all_game_info = pd.read_csv('all_games_04_on.csv').loc[:,'team':]


#restrict to 2017-2018 season
season_year_bool = all_game_info['season_end_year'] == 2018

#restrict to Warriors games
team_bool = all_game_info['team'] == 'gs'

all_game_info_gs_18 = all_game_info[season_year_bool & team_bool]

print(all_game_info_gs_18.columns.tolist())

all_game_info_gs_18.head()

['team', 'season_start_year', 'season_end_year', 'season_type', 'game_month', 'game_day', 'game_year', 'game_date', 'matchup_id']


,team,season_start_year,season_end_year,season_type,game_month,game_day,game_year,game_date,matchup_id
31746,gs,2017,2018,regular,10,17,2017,10/17/2017,400974438
31747,gs,2017,2018,regular,10,20,2017,10/20/2017,400974444
31748,gs,2017,2018,regular,10,21,2017,10/21/2017,400974784
31749,gs,2017,2018,regular,10,23,2017,10/23/2017,400974796
31750,gs,2017,2018,regular,10,25,2017,10/25/2017,400974814


In [3]:
#import player stats from 2017-2018 season
all_player_stats = pd.read_csv('players_stats_2018_cleaned.csv').loc[:,'team_name':]

#column names
print(all_player_stats.columns.tolist())

all_player_stats.head()

['team_name', 'player_name', 'player_id', 'position', 'started', 'played', 'minutes', 'fg_made', 'fg_attempted', 'three_pt_made', 'three_pt_attempted', 'ft_made', 'ft_attempted', 'oreb', 'dreb', 'reb', 'ast', 'stl', 'blk', 'to', 'pf', 'plusminus', 'pts', 'matchup_id']


,team_name,player_name,player_id,position,started,played,minutes,fg_made,fg_attempted,three_pt_made,three_pt_attempted,ft_made,ft_attempted,oreb,dreb,reb,ast,stl,blk,to,pf,plusminus,pts,matchup_id
0,OKC,carmelo-anthony,1975,PF,yes,yes,32,3,6,3,3,0,0,1,2,3,2,2,1,0,2,12,9,400975872
1,OKC,corey-brewer,3191,SF,yes,yes,29,5,8,1,4,3,4,1,1,2,3,2,0,1,4,6,14,400975872
2,OKC,paul-george,4251,SF,yes,yes,39,9,20,3,6,5,5,0,7,7,6,4,0,3,3,5,26,400975872
3,OKC,steven-adams,2991235,C,yes,yes,37,5,11,0,0,0,2,5,8,13,1,0,0,2,3,10,10,400975872
4,OKC,russell-westbrook,3468,PG,yes,yes,38,7,19,0,4,5,8,1,10,11,5,3,1,7,5,7,19,400975872


In [4]:
home_dir = !echo $HOME

#Define data directory
database_dir = home_dir[0] + '/database'

print(f'Database will persist at {database_dir}\n')

Database will persist at /Users/derekjung/database



In [5]:
%%bash -s "$database_dir"

#passed Python variable, later accessed with $1

#check if directory exists
if [ -d "$1" ] ; then

    echo "Directory already exists."

else
    #otherwise grapb file from Internet and store locally in data directory
    
    mkdir $1
    echo "creating database directory"

fi

Directory already exists.


In [6]:
con = sqlite3.connect("stats_June19_V1.db")

cur = con.cursor()

In [7]:
all_game_info_gs_18.to_sql(name='game_info_gs_18_tb', con=con, if_exists='replace',\
                          index=False, chunksize=1000)

#view some of game info table
sql_game_access = "\
SELECT * \
FROM game_info_gs_18_tb \
LIMIT 3 \
"

cur.execute(sql_game_access)

for row in cur:
    print(row)

('gs', 2017, 2018, 'regular', 10, 17, 2017, '10/17/2017', 400974438)
('gs', 2017, 2018, 'regular', 10, 20, 2017, '10/20/2017', 400974444)
('gs', 2017, 2018, 'regular', 10, 21, 2017, '10/21/2017', 400974784)


In [8]:
all_player_stats.to_sql(name='all_player_stats_tb', con=con, if_exists='replace',
                     index=False, chunksize=1000)

sql_stats_access = "\
SELECT * \
FROM all_player_stats_tb \
LIMIT 3 \
"

cur.execute(sql_stats_access)

for row in cur:
    print(row)


('OKC', 'carmelo-anthony', 1975, 'PF', 'yes', 'yes', 32, 3, 6, 3, 3, 0, 0, 1, 2, 3, 2, 2, 1, 0, 2, 12, 9, 400975872)
('OKC', 'corey-brewer', 3191, 'SF', 'yes', 'yes', 29, 5, 8, 1, 4, 3, 4, 1, 1, 2, 3, 2, 0, 1, 4, 6, 14, 400975872)
('OKC', 'paul-george', 4251, 'SF', 'yes', 'yes', 39, 9, 20, 3, 6, 5, 5, 0, 7, 7, 6, 4, 0, 3, 3, 5, 26, 400975872)


In [9]:
sql_player_stats_join = "\
SELECT DISTINCT  game_tb.season_end_year, game_tb.season_type,\
game_tb.game_date, game_tb.matchup_id AS game_matchup_id, \
player_tb.*\
FROM game_info_gs_18_tb AS game_tb \
JOIN all_player_stats_tb AS player_tb \
ON game_tb.matchup_id = player_tb.matchup_id \
"

player_stats_gs_18 = pd.read_sql(sql_player_stats_join, con)

player_stats_gs_18.head(30)

,season_end_year,season_type,game_date,game_matchup_id,team_name,player_name,player_id,position,started,played,minutes,fg_made,fg_attempted,three_pt_made,three_pt_attempted,ft_made,ft_attempted,oreb,dreb,reb,ast,stl,blk,to,pf,plusminus,pts,matchup_id
0,2018,regular,10/17/2017,400974438,GS,david-west,2177,PF,no,yes,9,2,3,0,0,0,0,1,0,1,0,0,1,1,0,0,4,400974438
1,2018,regular,10/17/2017,400974438,GS,draymond-green,6589,PF,yes,yes,28,2,6,1,2,4,4,1,10,11,13,0,0,2,3,7,9,400974438
2,2018,regular,10/17/2017,400974438,GS,javale-mcgee,3452,C,no,no,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,400974438
3,2018,regular,10/17/2017,400974438,GS,jordan-bell,3064427,C,no,yes,12,4,5,0,0,0,0,0,1,1,1,0,0,1,4,-2,8,400974438
4,2018,regular,10/17/2017,400974438,GS,kevin-durant,3202,SF,yes,yes,38,7,15,2,5,4,5,1,4,5,7,0,4,8,4,11,20,400974438
5,2018,regular,10/17/2017,400974438,GS,kevon-looney,3155535,SF,no,yes,8,0,0,0,0,1,2,0,2,2,0,1,0,0,2,-7,1,400974438
6,2018,regular,10/17/2017,400974438,GS,klay-thompson,6475,SG,yes,yes,38,6,14,4,7,0,0,0,6,6,3,2,2,1,0,0,16,400974438
7,2018,regular,10/17/2017,400974438,GS,nick-young,3243,SG,no,yes,26,8,9,6,7,1,1,1,1,2,0,0,0,1,4,-10,23,400974438
8,2018,regular,10/17/2017,400974438,GS,omri-casspi,3554,F,no,yes,4,0,0,0,0,2,2,0,1,1,0,0,0,0,0,-4,2,400974438
9,2018,regular,10/17/2017,400974438,GS,patrick-mccaw,3137730,SG,no,yes,19,2,3,0,0,0,0,1,2,3,1,1,1,0,1,-9,4,400974438


In [10]:
'''
#split up DataFrame in Warriors stats and opponents stats
player_stats_gs_18_gs = player_stats_gs_18[player_stats_gs_18['team_name']=='GS']
player_stats_gs_18_opponents = player_stats_gs_18[player_stats_gs_18['team_name']!='GS']

'''

"\n#split up DataFrame in Warriors stats and opponents stats\nplayer_stats_gs_18_gs = player_stats_gs_18[player_stats_gs_18['team_name']=='GS']\nplayer_stats_gs_18_opponents = player_stats_gs_18[player_stats_gs_18['team_name']!='GS']\n\n"

In [11]:
'''
del player_stats_gs_18_gs['matchup_id']

print(player_stats_gs_18_gs.columns.tolist())

player_stats_gs_18_gs.head()
'''

"\ndel player_stats_gs_18_gs['matchup_id']\n\nprint(player_stats_gs_18_gs.columns.tolist())\n\nplayer_stats_gs_18_gs.head()\n"

In [12]:
'''
del player_stats_gs_18_opponents['matchup_id']

player_stats_gs_18_opponents.head()
'''

"\ndel player_stats_gs_18_opponents['matchup_id']\n\nplayer_stats_gs_18_opponents.head()\n"

## Calculating Hollinger Game Score

We now append an additional column to each DataFrame, which is the game score for that player. We starting defining a function that calculates the game score for a player. Recall the Hollinger Game Score is given by

$
PTS + 0.4 * FG - 0.7 * FGA - 0.4*(FTA - FT) + 0.7 * ORB + 0.3 * DRB + STL + 0.7 * AST + 0.7 * BLK - 0.4 * PF - TOV,
$

with abbreviations described before.

In [13]:
#print(player_stats_gs_18_gs.columns.tolist())

In [14]:
def hollinger_game_score(row):
    '''
    Calculates Hollinger Game Score for a player's performance in a game.
    
    Input:
    row: row of DataFrame
    
    Output:
    float: game score for player
    '''
    
    game_score = row['pts'] + 0.4*row['fg_made'] - 0.7*row['fg_attempted'] \
                - 0.4*(row['ft_attempted']-row['ft_made']) + 0.7*row['oreb'] \
                + 0.3*row['dreb'] + row['stl'] + 0.7*row['ast'] + 0.7*row['blk'] \
                -0.4*row['pf'] - row['to']
            
    return game_score    

In [15]:
player_stats_gs_18.loc[:,'game_score'] = player_stats_gs_18.apply(hollinger_game_score, axis=1)
#player_stats_gs_18_opponents.loc[:,'game_score'] = player_stats_gs_18_opponents.apply(hollinger_game_score, axis=1)

In [16]:
player_stats_gs_18.head()

,season_end_year,season_type,game_date,game_matchup_id,team_name,player_name,player_id,position,started,played,minutes,fg_made,fg_attempted,three_pt_made,three_pt_attempted,ft_made,ft_attempted,oreb,dreb,reb,ast,stl,blk,to,pf,plusminus,pts,matchup_id,game_score
0,2018,regular,10/17/2017,400974438,GS,david-west,2177,PF,no,yes,9,2,3,0,0,0,0,1,0,1,0,0,1,1,0,0,4,400974438,3.1
1,2018,regular,10/17/2017,400974438,GS,draymond-green,6589,PF,yes,yes,28,2,6,1,2,4,4,1,10,11,13,0,0,2,3,7,9,400974438,15.2
2,2018,regular,10/17/2017,400974438,GS,javale-mcgee,3452,C,no,no,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,400974438,0.0
3,2018,regular,10/17/2017,400974438,GS,jordan-bell,3064427,C,no,yes,12,4,5,0,0,0,0,0,1,1,1,0,0,1,4,-2,8,400974438,4.5
4,2018,regular,10/17/2017,400974438,GS,kevin-durant,3202,SF,yes,yes,38,7,15,2,5,4,5,1,4,5,7,0,4,8,4,11,20,400974438,11.9


In [17]:
#player_stats_gs_18_opponents.head()

## Using max game score for classification

We now test the following hypothesis:

<center> For each game, the player with the highest Hollinger Game Score typically wins. <br>
    
We will test this on the Warriors 2017-2018 season. 

We preface this section by recalling that classifying all games as 'win' is successful $71.8\%$ of the time.

In [18]:
#store player stats from all Warriors in 2017-2018 season
player_stats_gs_18.to_sql(name='player_stats_gs_tb', con=con, if_exists='replace',
                     index=False, chunksize=1000)

sql_stats_access = "\
SELECT * \
FROM player_stats_gs_tb \
LIMIT 3 \
"

cur.execute(sql_stats_access)

for row in cur:
    print(row)

(2018, 'regular', '10/17/2017', 400974438, 'GS', 'david-west', 2177, 'PF', 'no', 'yes', 9, 2, 3, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 4, 400974438, 3.1000000000000005)
(2018, 'regular', '10/17/2017', 400974438, 'GS', 'draymond-green', 6589, 'PF', 'yes', 'yes', 28, 2, 6, 1, 2, 4, 4, 1, 10, 11, 13, 0, 0, 2, 3, 7, 9, 400974438, 15.2)
(2018, 'regular', '10/17/2017', 400974438, 'GS', 'javale-mcgee', 3452, 'C', 'no', 'no', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 400974438, 0.0)


In [19]:
#import team data for all games since 2009-2010 season 
all_team_stats = pd.read_csv('all_team_stats_2009_to_2018.csv').loc[:,'team':]


all_team_stats.to_sql(name='all_team_stats_tb', con=con, if_exists='replace',
                     index=False, chunksize=1000)

sql_game_stats_join  = "\
SELECT DISTINCT game_tb.season_end_year, game_tb.season_type,\
game_tb.game_date, game_tb.matchup_id AS game_matchup_id, \
team_tb.*\
FROM game_info_gs_18_tb AS game_tb \
JOIN all_team_stats_tb AS team_tb \
ON game_tb.matchup_id = team_tb.matchup_id \
"

team_stats_gs_18 = pd.read_sql(sql_game_stats_join, con)

team_stats_gs_18_gs = team_stats_gs_18[team_stats_gs_18['team']=='GS']


team_stats_gs_18_gs.to_sql(name='team_stats_gs_tb', con=con, if_exists='replace',
                     index=False, chunksize=1000)

team_stats_gs_18.head()

,season_end_year,season_type,game_date,game_matchup_id,team,first_qtr_points,second_qtr_points,third_qtr_points,fourth_qtr_points,total_points,fg_made,fg_attempted,fg_percentage,threept_made,threept_attempted,threept_percentage,ft_made,ft_attempted,ft_percentage,total_rebounds,offensive_rebounds,defensive_rebounds,assists,steals,blocks,total_turnovers,points_off_turnovers,fast_break_points,points_in_paint,personal_fouls,technical_fouls,flagrant_fouls,number_of_ot_periods,ot_points,won,away_or_home,matchup_id
0,2018,regular,10/17/2017,400974438,GS,35,36,30,20,121,43,80,53.8,16,30,53.3,19,21,90.5,42,6,35,34,5,9,17,21,36,32,25,0,0,0,[],0,Home,400974438
1,2018,regular,10/17/2017,400974438,HOU,34,28,26,34,122,47,97,48.5,15,41,36.6,13,19,68.4,53,10,33,28,9,5,13,11,15,54,16,1,1,0,[],1,Away,400974438
2,2018,regular,10/20/2017,400974444,GS,26,35,37,30,128,47,92,51.1,18,41,43.9,16,20,80.0,53,10,39,29,6,8,18,28,23,30,23,1,0,0,[],1,Away,400974444
3,2018,regular,10/20/2017,400974444,NO,39,25,26,30,120,45,98,45.9,16,36,44.4,14,22,63.6,57,16,35,26,8,4,19,18,14,32,19,1,0,0,[],0,Home,400974444
4,2018,regular,10/21/2017,400974784,GS,26,25,20,30,101,33,84,39.3,12,38,31.6,23,27,85.2,55,12,34,20,10,7,17,24,32,36,28,3,0,0,[],0,Away,400974784


In [22]:
#find player with max score for each game
#small hiccup if players on different teams have highest max game score, but ignore for now
#attach actual stat if Warriors won
sql_max_game_score = "\
SELECT players.game_matchup_id, \
teams.game_date, \
players.team_name, \
players.player_name, \
MAX(players.game_score) AS highest_g_score, \
teams.won \
FROM player_stats_gs_tb AS players \
JOIN team_stats_gs_tb AS teams \
ON players.game_matchup_id = teams.matchup_id \
GROUP BY players.game_matchup_id \
"

cur.execute(sql_max_game_score)

gs_max_game_score = pd.read_sql(sql_max_game_score, con)

gs_max_game_score.head()

,game_matchup_id,game_date,team_name,player_name,highest_g_score,won
0,400974438,10/17/2017,HOU,james-harden,21.6,0
1,400974444,10/20/2017,NO,anthony-davis,34.4,1
2,400974446,12/25/2017,CLE,kevin-love,22.1,1
3,400974784,10/21/2017,GS,stephen-curry,28.6,0
4,400974796,10/23/2017,GS,stephen-curry,25.9,1


In [23]:
#add column of 1 if GS had player with highest game score, 
#  0 if other team had player with highest game score

def gs_or_other_max_game_score(row):
    if row['team_name'] == 'GS':
        return 1
    else:
        return 0
    
gs_max_game_score['pred_gs_won'] = gs_max_game_score.apply(gs_or_other_max_game_score, axis=1)

gs_max_game_score.head()

,game_matchup_id,game_date,team_name,player_name,highest_g_score,won,pred_gs_won
0,400974438,10/17/2017,HOU,james-harden,21.6,0,0
1,400974444,10/20/2017,NO,anthony-davis,34.4,1,0
2,400974446,12/25/2017,CLE,kevin-love,22.1,1,0
3,400974784,10/21/2017,GS,stephen-curry,28.6,0,1
4,400974796,10/23/2017,GS,stephen-curry,25.9,1,1


We now need to attach a column that states whether the Warriors actually won the game. We then compute the accuracy of choosing the max score.

In [27]:
from sklearn.metrics import accuracy_score, classification_report

score = accuracy_score(gs_max_game_score.loc[:,'won'], gs_max_game_score.loc[:,'pred_gs_won'])

print('Accuracy by max game score: {0}%'.format(str(100*score))) 

print(classification_report(gs_max_game_score.loc[:,'won'], gs_max_game_score.loc[:,'pred_gs_won']))

Accuracy by max game score: 72.8155339806%
             precision    recall  f1-score   support

          0       0.51      0.66      0.58        29
          1       0.85      0.76      0.80        74

avg / total       0.75      0.73      0.74       103



We find that the max game score is a pretty poor prediction, with an accuracy very similar to that of classifying as all 'win'. We conclude this notebook by comparing this with the strategy of choosing sum of game score instead of max game score. Note that if we divide this sum by 5, we arrive at the average game score of a player on the court at all times.

In [31]:
#find player with average score for each game
#attach actual stat if Warriors won
sql_sum_game_score = "\
SELECT players.game_matchup_id, \
teams.game_date, \
players.team_name, \
SUM(players.game_score)/5 AS average_g_score, \
teams.won \
FROM player_stats_gs_tb AS players \
JOIN team_stats_gs_tb AS teams \
ON players.game_matchup_id = teams.matchup_id \
GROUP BY players.game_matchup_id, players.team_name \
"

cur.execute(sql_sum_game_score)

gs_sum_game_score = pd.read_sql(sql_sum_game_score, con)

gs_sum_game_score.head()

,game_matchup_id,game_date,team_name,average_g_score,won
0,400974438,10/17/2017,GS,20.84,0
1,400974438,10/17/2017,HOU,20.22,0
2,400974444,10/20/2017,GS,20.84,1
3,400974444,10/20/2017,NO,18.46,1
4,400974446,12/25/2017,CLE,11.48,1


In [32]:
#need to separate into two DataFrames

gs_sum_game_score_gs = gs_sum_game_score[gs_sum_game_score['team_name'] == 'GS']
gs_sum_game_score_opponents = gs_sum_game_score[gs_sum_game_score['team_name']!= 'GS'] 

In [33]:
gs_sum_game_score_gs.head()

,game_matchup_id,game_date,team_name,average_g_score,won
0,400974438,10/17/2017,GS,20.84,0
2,400974444,10/20/2017,GS,20.84,1
5,400974446,12/25/2017,GS,16.52,1
6,400974784,10/21/2017,GS,14.62,0
9,400974796,10/23/2017,GS,23.78,1


In [34]:
gs_sum_game_score_opponents.head()

,game_matchup_id,game_date,team_name,average_g_score,won
1,400974438,10/17/2017,HOU,20.22,0
3,400974444,10/20/2017,NO,18.46,1
4,400974446,12/25/2017,CLE,11.48,1
7,400974784,10/21/2017,MEM,16.70,0
8,400974796,10/23/2017,DAL,14.02,1


It makes sense that game score is usually between 13 and 20. Recall that Game Score measures performance similar to points scored. Since teams usually score in the neighborhood of 100 points (+/- 20 points), each player should have a game score of a fifth of this.

We now make a series of which team scored more.

In [62]:
def list_greater(main_column, opponent_column):
    '''
    Return list that states which team has the greater particular stat (1 if main team, 0 if opponent team).
    
    Input:
    main_row, opponent_row: two Series of numbers of equal length
    
    Output:
    list of 0's and 1's
    '''
    
    main_list = main_column.tolist()
    opponent_list = opponent_column.tolist()
    
    #1 if main team has greater stat
    #0 if opponent team has smaller stat
    comp_list = []
    
    for idx in range(len(main_list)):
        if main_list[idx] >= opponent_list[idx]:
            comp_list.append(1)
        
        elif opponent_list[idx] > main_list[idx]:
            comp_list.append(0)
        
    return comp_list

In [68]:
comp_average_g_score = list_greater(gs_sum_game_score_gs.loc[:,'average_g_score'],gs_sum_game_score_opponents.loc[:,'average_g_score'])


score = accuracy_score(comp_average_g_score, gs_sum_game_score_gs.loc[:,'won'].tolist())

print('Accuracy by average game score: {0:2f}%'.format(100*score))

print(classification_report(comp_average_g_score, gs_sum_game_score_gs.loc[:,'won'].tolist()))

Accuracy by average game score: 97.087379%
             precision    recall  f1-score   support

          0       0.90      1.00      0.95        26
          1       1.00      0.96      0.98        77

avg / total       0.97      0.97      0.97       103



Wow, that's tremendous! This stat that takes into account many stats (more than just points) predicting winning with $97\%$ accuracy! We will need to study this more in a future notebook.